In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings('ignore')

WARNING : Poor documentation  ( This is supposed to be a supporting script for the original notebook )
Original notebook - https://www.kaggle.com/tomtillo/whom-should-you-interview-next

* How did we approach it : We know which Heroes' youtube videos gets more views - 
* We might have a hunch that it might be due to their ranking in kaggle . 
* Here , we try to find similar potential heroes - with similar "kaggle properties" as the next candidates 



In [ ]:
df_episodes = pd.read_csv('/kaggle/input/chai-time-data-science/Episodes.csv')   #  Episodes 
df_users = pd.read_csv('/kaggle/input/meta-kaggle/Users.csv')
df_achieve = pd.read_csv('/kaggle/input/meta-kaggle/UserAchievements.csv')

In [ ]:
df_episodes = pd.read_csv('/kaggle/input/chai-time-data-science/Episodes.csv')   #  Episodes 

In [ ]:
df_interest = df_episodes[['youtube_views', 'youtube_watch_hours', 'heroes_kaggle_username']]

Build your 'interested' df - The dataset that we are interested in !

In [ ]:
df_interest.head()

Remove one user - they are a bunch too much

In [ ]:
df_interest  = pd.DataFrame(df_interest[~df_interest['heroes_kaggle_username'].isna()])
df_interest =  df_interest[df_interest['heroes_kaggle_username'] != 'dott1718 | philippsinger' ]

Add some more columns - join_date , user_id

In [ ]:
df_interest['kaggle_join_date'] = df_interest.heroes_kaggle_username.apply(lambda x : df_users[df_users['UserName'] == x].iloc[0,3])
df_interest['kaggle_userid'] = df_interest.heroes_kaggle_username.apply(lambda x : df_users[df_users['UserName'] == x].iloc[0,0])

In [ ]:
trunc = df_achieve[df_achieve.UserId.isin(df_interest['kaggle_userid'])]
trunc.head()

Add more columns - competition rank, discussion rank, script rank

In [ ]:
df_interest['high_comp'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Competitions')].iloc[0,7])
df_interest['high_disc'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Scripts')].iloc[0,7])
df_interest['high_scripts'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Discussion')].iloc[0,7])

Convert into months

In [ ]:
from datetime import date
from datetime import datetime

#function to return date from the date of joining kaggle
def get_months(str_d1):
    str_d2 = '07/12/2020'  # hardcode some date around  last month ( later take this as the date of interview)
    f_date = datetime.strptime(str_d1, '%m/%d/%Y')
    l_date = datetime.strptime(str_d2, '%m/%d/%Y')
    delta = l_date - f_date
    return  delta.days/30

#get_months(df_episodes['kaggle_join_date'].iloc[1])

df_interest['df_months_in_kaggle'] = df_interest['kaggle_join_date'].apply(lambda x :get_months(x))

Scale the columns - MinMax will do for now 

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

df_interest[['df_months_in_kaggle', 'youtube_views','high_comp','high_scripts','high_disc']] = \
scaler.fit_transform(df_interest[['df_months_in_kaggle', 'youtube_views','high_comp','high_scripts','high_disc']])

If there are NaN in the values, replace with the highest number 

In [ ]:
df_interest['high_comp'][df_interest.high_comp.isna()] = df_interest.high_comp.max()
df_interest['high_disc'][df_interest.high_disc.isna()] = df_interest.high_disc.max()
df_interest['high_scripts'][df_interest.high_scripts.isna()] =df_interest.high_scripts.max()

In [ ]:
df_interest['high_comp'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Competitions')].iloc[0,7])
df_interest['high_disc'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Scripts')].iloc[0,7])
df_interest['high_scripts'] = df_interest.kaggle_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Discussion')].iloc[0,7])

Pull top 200 Discussion ranks into the pool 

In [ ]:
df_pool = df_achieve[df_achieve['AchievementType'] =='Discussion'].sort_values(by='HighestRanking').head(200)

Pull top 200 Scripts ranks into the pool 

In [ ]:
df_pool  = df_pool.append(df_achieve[df_achieve['AchievementType'] =='Scripts'].sort_values(by='HighestRanking').head(200))

Pull top 400  Competition ranks into the pool 

In [ ]:
df_pool =df_pool.append(df_achieve[df_achieve['AchievementType'] =='Competitions'].sort_values(by='HighestRanking').head(400))

In [ ]:
len(df_pool)

In [ ]:
df_pool.head()

In [ ]:
df_pool_userid = df_pool.UserId.unique()

In [ ]:
len(df_pool_userid)

In [ ]:
df_interest.head()

In [ ]:
df_pool_matrix = pd.DataFrame()
df_pool_matrix['pool_userid'] = df_pool_userid
df_pool_matrix['pool_username'] = df_pool_matrix.pool_userid.apply(lambda x : df_users[df_users['Id'] == x].iloc[0,1])
df_pool_matrix['kaggle_join_date'] = df_pool_matrix.pool_username.apply(lambda x : df_users[df_users['UserName'] == x].iloc[0,3])

In [ ]:
trunc = df_achieve[df_achieve.UserId.isin(df_pool_matrix['pool_userid'])]

In [ ]:
df_pool_matrix['high_comp'] = df_pool_matrix.pool_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Competitions')].iloc[0,7])
df_pool_matrix['high_disc'] = df_pool_matrix.pool_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Scripts')].iloc[0,7])
df_pool_matrix['high_scripts'] = df_pool_matrix.pool_userid.apply(lambda x : trunc[(trunc['UserId'] == x) & (trunc['AchievementType']=='Discussion')].iloc[0,7])

In [ ]:
from datetime import date
from datetime import datetime

#function to return date from the date of joining kaggle
def get_months(str_d1):
    str_d2 = '07/12/2020'  # hardcode some date around  last month ( later take this as the date of interview)
    f_date = datetime.strptime(str_d1, '%m/%d/%Y')
    l_date = datetime.strptime(str_d2, '%m/%d/%Y')
    delta = l_date - f_date
    return  delta.days/30

df_pool_matrix['df_months_in_kaggle'] = df_pool_matrix['kaggle_join_date'].apply(lambda x :get_months(x))

In [ ]:
df_pool_matrix.head()

In [ ]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

df_pool_matrix[['df_months_in_kaggle','high_comp','high_scripts','high_disc']] = \
scaler.fit_transform(df_pool_matrix[['df_months_in_kaggle','high_comp','high_scripts','high_disc']])

In [ ]:
df_pool_matrix['high_comp'][df_pool_matrix.high_comp.isna()] = df_pool_matrix.high_comp.max()
df_pool_matrix['high_disc'][df_pool_matrix.high_disc.isna()] = df_pool_matrix.high_disc.max()
df_pool_matrix['high_scripts'][df_pool_matrix.high_scripts.isna()] =df_pool_matrix.high_scripts.max()

In [ ]:
df_pool_matrix.head()
df_interest.sort_values(by='youtube_views',ascending=False).head(10)

In [ ]:
common_names = set(df_pool_matrix['pool_username']).intersection(set(df_interest['heroes_kaggle_username']))

In [ ]:
len(common_names)

In [ ]:
df_pool_matrix = df_pool_matrix[~df_pool_matrix['pool_username'].isin(common_names)]

### Actual Testing - 
How to - hardcode the kaggle userid for whom you want to see the similar other kagglers ( make sure the userid you provide is in the list of ctds heroes who have a kaggle id - refer to the code above for explanation)

In [ ]:
from scipy import spatial

heroes_user_name = "jhoward"

a = df_interest['high_comp'][df_interest['heroes_kaggle_username'] == heroes_user_name].iloc[0]
b = df_interest['high_disc'][df_interest['heroes_kaggle_username'] == heroes_user_name].iloc[0]
c = df_interest['high_scripts'][df_interest['heroes_kaggle_username'] == heroes_user_name].iloc[0]
d = df_interest['df_months_in_kaggle'][df_interest['heroes_kaggle_username'] == heroes_user_name].iloc[0]
hero_vector   = [a,b,c,d]
def get_distance(e,f,g,h,hero_vector=hero_vector ):
    pool_vector = [e,f,g,h]
    result = 1 - spatial.distance.cosine(hero_vector, pool_vector)
    return result

In [ ]:
df_pool_matrix['cosine_dist']= df_pool_matrix.apply(lambda x : get_distance(x['high_comp'],x['high_disc'],x['high_scripts'],x['df_months_in_kaggle']),axis = 1)
df_pool_matrix.sort_values(by='cosine_dist').head(5)